In [1]:
import pandas as pd
import numpy as np

In [13]:
#import data
suffix_no_nuclear = 'no-nuclear_no_coal_renewables'
suffix_normal = 'normal'

gen = pd.read_csv(f'./good_model_inputs/inputs_gen_{suffix_no_nuclear}.csv')
load = pd.read_csv(f'./good_model_inputs/inputs_load_no-nuclear.csv')
rencf = pd.read_csv(f'./good_model_inputs/inputs_renewableCF.csv')
trans = pd.read_csv(f'./good_model_inputs/inputs_trans_no-nuclear.csv')

In [19]:
rencf

,r,d,h,solarCF,windCF,t
0,ERC_REST,1,1,0.000031,0.595,1
1,ERC_REST,1,2,0.000031,0.563,2
2,ERC_REST,1,3,0.000031,0.563,3
3,ERC_REST,1,4,0.000031,0.563,4
4,ERC_REST,1,5,0.342211,0.563,5
...,...,...,...,...,...,...
525594,WEC_SDGE,365,19,0.017689,0.529,8755
525595,WEC_SDGE,365,20,0.017689,0.600,8756
525596,WEC_SDGE,365,21,0.017689,0.650,8757
525597,WEC_SDGE,365,22,0.017689,0.680,8758


In [14]:
#create a list of our regions
regions = set(load['r'])

In [15]:
#calculate regions where generation > load
for reg in regions:
    gn = np.zeros((8759,))
    gn += gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] != 'Solar').values * (gen['FuelType'] != 'Wind').values * (gen['FuelType'] != 'wind_generator').values * (gen['FuelType'] != 'solar_generator').values, 'Capacity'].values.sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Solar').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'solar_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'solarCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'Wind').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()
    gn += (gen.loc[(gen['RegionName'] == reg).values * (gen['FuelType'] == 'wind_generator').values, 'Capacity'].values.sum() * rencf.loc[rencf['r'] == reg, 'windCF'].values).sum()

    if (load.loc[load['r'] == reg, 'demandLoad'] > gn).values.sum() > 0:
        print(f'Generation cannot match load in region {reg}')
        print(max(load.loc[load['r'] == reg, 'demandLoad'] - gn))
    

In [16]:
#find the total generation for the region where generation > load

region = 'ERC_REST'
t = 5080

print( 'Nuclear generation ' + str(gen.loc[gen['RegionName'] == region].loc[gen['FuelType'] == 'Nuclear','Capacity'].sum()))

print( 'Total generation ' + str(gen.loc[gen['RegionName'] == region,'Capacity'].sum()))

#look at the transmission into the region where generation > load

print( 'Transmission into the region ' + str(trans.loc[(trans['r2'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()*.972))

print( 'Transmission out of the region ' + str(trans.loc[(trans['r1'] == region).values * (trans['transCap'] > 1).values, 'transCap'].sum()))

#find the total amount of energy needed to/from that region if all used

print( f'Maximum net energy left after use from the region at time {t} ' + 
    str(trans.loc[(trans['r2'] == region).values, 'transCap'].sum()*.972 + 
       gen.loc[gen['RegionName'] == region,'Capacity'].sum() - 
        trans.loc[(trans['r1'] == region).values, 'transCap'].sum() - 
       load.loc[(load['r'] == 'NY_Z_GI')].loc[(load['t'] == t)]['demandLoad'].values))

Nuclear generation 0.0
Total generation 91309.1
Transmission into the region 11916.72
Transmission out of the region 5529
Maximum net energy left after use from the region at time 5080 [95072.832]


In [17]:
#find what time stamps are associated with demand > load 
#(we will go back and manually change our load based on this, assuming some of the region blacks out)
load.loc[load['r'] == region].loc[load['t'].isin(range(5001,6000))].loc[load['demandLoad'] >= gen.loc[gen['RegionName'] == region,'Capacity'].sum()]

,Unnamed: 0,r,t,demandLoad


In [18]:
print( 'Solar generation ' + str(gen.loc[gen['FuelType'] == 'Solar','Capacity'].sum()))

print( 'Wind generation ' + str(gen.loc[gen['FuelType'] == 'Wind','Capacity'].sum()))


Solar generation 10842.0
Wind generation 68058.0
